In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
import plotly.express as px

## 客戶選址分析思路

In [2]:
df = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\即時路況\臺北市車輛偵測器即時路況歷史資料2023-07-18.CSV")

In [3]:
df.columns

Index(['VDID', 'HostAuthorityCode', 'AuthorityCode', 'SubAuthorityCode',
       'LinkID', 'LaneID', 'LaneType', 'LaneSpeed', 'Occupany', 'VehicleType',
       'VehicleVolume', 'VehicleSpeed', 'ErrTp', 'RecurrentTimes',
       'RecurrentZeroTimes', 'Status', 'DataCollectTime', 'SrcUpdateTime',
       'UpdateTime', 'InfoTime', 'InfoDate'],
      dtype='object')

In [4]:
# 將設備狀態正常的和車輛類型是小型車還有車輛數是>0的篩選出來
df = df[(df["Status"] == "正常") & (df["VehicleType"] == "S") & (df["VehicleVolume"] > 0)]

In [5]:
df.shape

(1950479, 21)

In [6]:
# 將資料收集時間轉為時間日期格式
df["DataCollectTime"] = pd.to_datetime(df["DataCollectTime"])

In [7]:
ref = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\數據\TDX\即時路況\臺北市車輛偵測器歷史資料2023-07-17.CSV")

In [8]:
ref

,VDID,AuthorityCode,SubAuthorityCode,BiDirectional,LinkID,Bearing,RoadDirection,LaneNum,ActualLaneNum,VDType,LocationType,DetectionType,PositionLon,PositionLat,Geohash,CountyID,CountyName,TownID,TownName,RoadID,RoadName,RoadClass,RoadSectionStart,RoadSectionEnd,LocationMile,SrcUpdateTime,UpdateTime,InfoTime,InfoDate
0,V1NMM20,TPE,NaN,偵測雙向,6004570600020A,W,W,2,2,線圈式,NaN,高快速公路_市快主線,121.54813,24.98710,wsqqjuu8,63000,臺北市,8,文山區,600457A,木柵路一段,NaN,NaN,NaN,NaN,2023-07-17 00:00:00+08:00,2023-07-18 01:45:44+08:00,2023-07-17 00:00:00+08:00,2023-07-17
1,V1NMM20,TPE,NaN,偵測雙向,6004570200020A,E,E,2,2,線圈式,NaN,高快速公路_市快主線,121.54813,24.98710,wsqqjuu8,63000,臺北市,8,文山區,600457A,木柵路一段,NaN,NaN,NaN,NaN,2023-07-17 00:00:00+08:00,2023-07-18 01:45:44+08:00,2023-07-17 00:00:00+08:00,2023-07-17
2,V1010A0,TPE,NaN,偵測單向,2000400100400A,N,N,3,3,線圈式,NaN,高快速公路_市快主線,121.53778,25.02761,wsqqmsjh,63000,臺北市,3,大安區,200040,建國南北快速道路,NaN,NaN,NaN,NaN,2023-07-17 00:00:00+08:00,2023-07-18 01:45:44+08:00,2023-07-17 00:00:00+08:00,2023-07-17
3,V0130A0,TPE,NaN,偵測單向,2000201002000A,E,E,3,3,線圈式,NaN,高快速公路_市快主線,121.51876,25.04831,wsqqmpzk,63000,臺北市,5,中正區,200020,市民大道高架道路,NaN,NaN,NaN,NaN,2023-07-17 00:00:00+08:00,2023-07-18 01:45:44+08:00,2023-07-17 00:00:00+08:00,2023-07-17
4,V0380F0,TPE,NaN,偵測單向,2000200100610A,SW,W,2,2,線圈式,NaN,高快速公路_市快主線,121.56620,25.04546,wsqqqr3u,63000,臺北市,2,信義區,200020,市民大道高架道路,NaN,NaN,NaN,NaN,2023-07-17 00:00:00+08:00,2023-07-18 01:45:44+08:00,2023-07-17 00:00:00+08:00,2023-07-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,VSMPW60,TPE,NaN,偵測雙向,6005120200030A,E,E,2,2,微波式,NaN,其它道路路口_靠近路口或停止線,121.55623,25.07994,wsqqwjdt,63000,臺北市,4,中山區,600512A,樂群二路,NaN,NaN,NaN,NaN,2023-07-17 00:00:00+08:00,2023-07-18 01:45:44+08:00,2023-07-17 00:00:00+08:00,2023-07-17
966,VVVGB40,TPE,NaN,偵測雙向,6003750700000A,NW,NW,4,4,線圈式,NaN,高快速公路_市快主線,121.51434,25.09452,wsqqv0k4,63000,臺北市,11,士林區,600375A,承德路五段,NaN,NaN,NaN,NaN,2023-07-17 00:00:00+08:00,2023-07-18 01:45:44+08:00,2023-07-17 00:00:00+08:00,2023-07-17
967,VVVGB40,TPE,NaN,偵測雙向,6003750300000A,SE,SE,2,2,線圈式,NaN,高快速公路_市快主線,121.51434,25.09452,wsqqv0k4,63000,臺北市,11,士林區,600375A,承德路五段,NaN,NaN,NaN,NaN,2023-07-17 00:00:00+08:00,2023-07-18 01:45:44+08:00,2023-07-17 00:00:00+08:00,2023-07-17
968,VZLKH60,TPE,NaN,偵測雙向,6003260200020A,E,E,1,1,線圈式,NaN,高快速公路_市快主線,121.53722,25.11057,wsqqvehx,63000,臺北市,11,士林區,600326A,德行東路,NaN,NaN,NaN,NaN,2023-07-17 00:00:00+08:00,2023-07-18 01:45:44+08:00,2023-07-17 00:00:00+08:00,2023-07-17


In [9]:
ref_dict = dict(zip(ref["LinkID"], ref[["TownName", "RoadName"]].apply(tuple, axis=1)))

In [10]:
extract_df = pd.DataFrame(df["LinkID"].map(ref_dict).tolist(), index=df.index, columns=["TownName", "RoadName"])

In [11]:
df = pd.concat([df, extract_df], axis=1)

In [12]:
df = df.dropna(subset=["RoadName"])

In [13]:
df = df[(~df["RoadName"].str.contains("快速|高架|大道|引道"))]

In [14]:
df.shape

(1311528, 23)

In [15]:
result = df.groupby([pd.Grouper(key="DataCollectTime", freq="H"), "TownName", 'RoadName'])[["VehicleVolume"]].sum().reset_index()

In [16]:
plot = result.copy()
plot.shape

(5508, 4)

In [17]:
px.bar(plot.groupby(["DataCollectTime", "TownName"])[["VehicleVolume"]].sum().reset_index(), x="DataCollectTime", y="VehicleVolume", text_auto=True, color="TownName")

In [18]:
px.bar(plot.groupby(["DataCollectTime", "TownName"])[["VehicleVolume"]].sum().reset_index(), x="DataCollectTime",\
    y="VehicleVolume", facet_col="TownName", title="台北市各行政區車流量分佈", labels={"DataCollectTime": "時段", "TownName": "行政區", "VehicleVolume": "車流量"})

In [19]:
px.bar(plot.query("TownName in ['中山區', '內湖區', '士林區', '大安區']"), x="DataCollectTime", y="VehicleVolume", facet_col="TownName", color="RoadName", title="主要行政區車流量分佈", labels={"DataCollectTime": "時段", "TownName": "行政區", "VehicleVolume": "車流量", "RoadName": "路段名"})

In [20]:
car = pd.read_csv(r"D:\kc.hsu\OneDrive - Bridgestone\數據\監理站新車領牌數據\20221017_監理站領牌數據全.csv")

C:\Users\kc.hsu\AppData\Local\Temp\ipykernel_2596\2411190092.py:1: DtypeWarning:

Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.



In [21]:
car["車輛類型3"].unique()

array(['PSV', 'LCV', 'Truck/Bus'], dtype=object)

In [22]:
car = car[(car["城市"] == "臺北市") & (car["車輛類型3"] == "PSV")]

In [23]:
plt = car.groupby(["來源", "區"]).size().reset_index(name="數量").sort_values(by=["來源", "數量"], ascending=False)

In [24]:
plt

,來源,區,數量
12,進口,中山區,16157
15,進口,內湖區,14562
20,進口,大安區,13206
22,進口,松山區,7848
14,進口,信義區,5017
...,...,...,...
5,國產,南港區,2643
4,國產,北投區,2594
1,國產,中正區,2225
7,國產,大同區,1702


In [25]:
px.bar(plt, x="區", y="數量", color="來源")